In [ ]:
# Author: Mickey Huang
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def moving_average(actions, rolling_window=20) :
        ret = np.cumsum(actions, dtype=float)
        ret[rolling_window:] = ret[rolling_window:] - ret[:-rolling_window]
        return ret[rolling_window - 1:] / rolling_window
    
def load_data(filePath, name):
    data = pd.read_csv(filePath)
    data = data.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Adj Close': 'adjcp', 'Volume': 'volume'})
    data['tic'] = name
    return data

def plot(actions, rolling_window, train_start_date, train_end_date, test_start_date, test_end_date):
    if actions.shape[1] == 5: #with cash
        actions = actions[['date', 'TLT', 'VNQ', 'VTI', 'Cash']]
    elif actions.shape[1] == 4: #without cash
        actions = actions[['date', 'TLT', 'VNQ', 'VTI']]
    time_index = actions.date.values
    time_index = [np.datetime64(x) for x in time_index]
    
    plt.rcParams["figure.figsize"] = (15,6)
    plt.plot(time_index[rolling_window-1:], moving_average(actions.iloc[:, 1].values, rolling_window))
    plt.plot(time_index[rolling_window-1:], moving_average(actions.iloc[:, 2].values, rolling_window))
    plt.plot(time_index[rolling_window-1:], moving_average(actions.iloc[:, 3].values, rolling_window))
    plt.plot(time_index[rolling_window-1:], moving_average(actions.iloc[:, 4].values, rolling_window))
    plt.ylim(0, 0.6)
    plt.legend(['TLT','VNQ','VTI', 'Cash'])
    plt.axvline(x = np.datetime64(test_start_date), color = 'purple', label = 'axvline - full height')
    plt.title('weight of each component in period(' + train_start_date[:3] + '-' + test_start_date[:3] + ')')
    plt.xlabel('year')
    plt.ylabel('weights')
    #plt.savefig('weight of each component in period(' + train_start_date[:3] + '-' + test_start_date[:3] + ')')
    
    TLT = load_data('../data/TLT.csv', 'TLT')
    VTI = load_data('../data/VTI.csv', 'VTI')
    VNQ = load_data('../data/VNQ.csv', 'VNQ')
    tlt_close = TLT[(TLT.date>=train_start_date) & (TLT.date<test_end_date)].close.values[-actions.shape[0]:]
    vnq_close = VNQ[(VNQ.date>=train_start_date) & (VNQ.date<test_end_date)].close.values[-actions.shape[0]:]
    vti_close = VTI[(VTI.date>=train_start_date) & (VTI.date<test_end_date)].close.values[-actions.shape[0]:]
    plt.figure()
    plt.rcParams["figure.figsize"] = (15,6)
    plt.plot(time_index, tlt_close)
    plt.plot(time_index, vnq_close)
    plt.plot(time_index, vti_close)
    plt.legend(['TLT', 'VNQ', 'VTI'])
    plt.axvline(x = np.datetime64(test_start_date), color = 'purple', label = 'axvline - full height')
    plt.title('price trend of each component in period(' + train_start_date[:3] + '-' + test_start_date[:3] + ')')
    plt.xlabel('year')
    plt.ylabel('price')
    #lt.savefig('price trend of each component in period(' + train_start_date[:3] + '-' + test_start_date[:3] + ')')

In [ ]:
# example
action_path = '' # 請填入 action 路徑
actions = pd.read_csv(action_path)
plot(actions, 1, '2016-01-01', '2021-01-01', '2021-01-01', '2022-01-01')